In [1]:

import json
from src.base_creature_builder import creature_template_builder
from src.validator import validate_creature, finalization
from src.weapons_builder import weapon_template_builder
from src.utils import fill_template_with_openai, generate_id
from src.base_creature_builder import CREATURE_TEMPLATE_DICT
from src.validator import validate_weapon
    

In [2]:
final_creature_dict = CREATURE_TEMPLATE_DICT

In [3]:
creature_builder_string = creature_template_builder(
    creature_description="A goblin that throws pies at its enemies, it keeps the pies in a satchel at its side.",
    creature_name="Pie Goblin",
    creature_type="Minion",
    setting_description="A generic D&D-like fantasty setting.",
    skills=["Stealth", "Ranged (Light)"],
    allow_other_skills=True,
    combat_cr=2,
    social_cr=1,
    agility=3,
    cunning=3,
    presence=1)
generated_character_data = fill_template_with_openai(creature_builder_string)
generated_character_data_as_dict = validate_creature(generated_character_data)
final_creature_dict["characters"] = [generated_character_data_as_dict]
print(final_creature_dict)

passes all validation steps
{'customArchetypes': [{'name': 'NPC Archetype', 'Brawn': 1, 'Agility': 1, 'Intellect': 1, 'Cunning': 1, 'Willpower': 1, 'Presence': 1, 'woundThreshold': 5, 'strainThreshold': 1, 'experience': '1', 'description': '', 'setting': ['All'], 'skills': {}, 'talents': [], 'id': 'ByO058vfgGB33EOrabeE'}], 'customCareers': [{'name': 'NPC Career', 'description': '', 'setting': ['All'], 'skills': [], 'id': '9VWOCKS9chsyHcBjS1U3'}], 'characters': [{'name': 'Pie Goblin', 'careerSkillsRank': ['Stealth', 'RangedLight', 'Deception', 'Athletics'], 'masterSkills': {'Stealth': {'careerRank': 2}, 'Deception': {'careerRank': 1}, 'RangedLight': {'careerRank': 2}, 'Athletics': {'careerRank': 1}}, 'creationCharacteristics': {'Brawn': 1, 'Agility': 2, 'Intellect': 1, 'Cunning': 2, 'Willpower': 0, 'Presence': 0}, 'archetype': 'Npc', 'career': 'NpcCareer', 'theme': 'ROT'}]}


In [5]:
#final_data = finalization(generated_character_data_as_dict, None)

In [18]:
#with open("creature.json", "w") as f:
#    f.write(json.dumps(final_data))
#f.close()

In [4]:
creature_description="A goblin that throws pies at its enemies, it keeps the pies in a satchel at its side."
creature_name="Pie Goblin"
creature_type="Minion"
setting_description="A generic D&D-like fantasty setting."
creature_skills=["Stealth", "Ranged (Light)"],
combat_cr=2
weapon_builder_string = weapon_template_builder(creature_name, creature_description, creature_type, setting_description, creature_skills, combat_cr, number_of_weapons=2)

generated_weapon_data = fill_template_with_openai(weapon_builder_string)


In [7]:
print(generated_weapon_data)

[
    {
        "name": "Pie Satchel",
        "encumbrance": 2,
        "setting": [
            "All"
        ],
        "damage": "2",
        "range": "Short",
        "skill": "RangedLight",
        "critical": 5,
        "modifier": {},
        "qualities": {
            "LimitedAmmo": "5",
            "StunDamage": ""
        }
    },
    {
        "name": "Super Pie Launcher",
        "encumbrance": 4,
        "setting": [
            "All"
        ],
        "damage": "4",
        "range": "Medium",
        "skill": "RangedLight",
        "critical": 4,
        "modifier": {},
        "qualities": {
            "LimitedAmmo": "3",
            "Blast": "1",
            "StunDamage": ""
        }
    }
]


In [10]:
import ast
from src.validator import REQUIRED_CHARACTER_FIELDS, REQUIRED_WEAPON_FIELDS
def validate_weapon(generated_data):
    try:
        generated_data_as_dict = ast.literal_eval(generated_data)
    except Exception as e:
        print("cannot convert to dict")  # TODO: ask OpenAI to fix this
        raise e
    passes_all_validation_steps = True
    for each_weapon in generated_data_as_dict:
        for k in each_weapon.keys():
            if k not in REQUIRED_WEAPON_FIELDS:
                print("warning, extraneous field: ", k)
                passes_all_validation_steps = False
            
        for k in REQUIRED_WEAPON_FIELDS:
            if k not in each_weapon.keys():
                print("warning, missing field: ", k)
                passes_all_validation_steps = False
        if passes_all_validation_steps:
            print("passes all validation steps")
        
        each_weapon["id"] = generate_id()
    
    return generated_data_as_dict

In [19]:
generated_weapon_data_as_dict

[{'name': 'Pie Satchel',
  'encumbrance': 2,
  'setting': ['All'],
  'damage': '2',
  'range': 'Short',
  'skill': 'RangedLight',
  'critical': 5,
  'modifier': {},
  'qualities': {'LimitedAmmo': '5', 'StunDamage': ''},
  'id': 'BOgrmkaKLuQRwvhGBzIX'},
 {'name': 'Super Pie Launcher',
  'encumbrance': 4,
  'setting': ['All'],
  'damage': '4',
  'range': 'Medium',
  'skill': 'RangedLight',
  'critical': 4,
  'modifier': {},
  'qualities': {'LimitedAmmo': '3', 'Blast': '1', 'StunDamage': ''},
  'id': '4o59gUuKajWhtWIsn0EH'}]

In [15]:

generated_weapon_data_as_dict = validate_weapon(generated_weapon_data)
print(generated_weapon_data_as_dict)
final_creature_dict["customWeapons"] = generated_weapon_data_as_dict

passes all validation steps
passes all validation steps
[{'name': 'Pie Satchel', 'encumbrance': 2, 'setting': ['All'], 'damage': '2', 'range': 'Short', 'skill': 'RangedLight', 'critical': 5, 'modifier': {}, 'qualities': {'LimitedAmmo': '5', 'StunDamage': ''}, 'id': 'BOgrmkaKLuQRwvhGBzIX'}, {'name': 'Super Pie Launcher', 'encumbrance': 4, 'setting': ['All'], 'damage': '4', 'range': 'Medium', 'skill': 'RangedLight', 'critical': 4, 'modifier': {}, 'qualities': {'LimitedAmmo': '3', 'Blast': '1', 'StunDamage': ''}, 'id': '4o59gUuKajWhtWIsn0EH'}]


In [25]:
import re
final_creature_dict["characters"][0]["equipmentWeapons"] = {}
for weapon in generated_weapon_data_as_dict:
    new_name = weapon['name'].replace(' ', '')  # Remove spaces
    new_name = new_name[0].lower() + new_name[1:]  # Lowercase the first letter
    final_creature_dict["characters"][0]["equipmentWeapons"][weapon['id']] = {"craftsmanship": "",
                    "carried": True,
                    "id": new_name,
                    "equipped": True}

In [26]:
with open("creature.json", "w") as f:
   f.write(json.dumps(final_creature_dict))
f.close()

In [82]:
import requests

# Create a session
session = requests.Session()

In [84]:
response.json()

{'message': 'No API key found in request',
 'hint': 'No `apikey` request header or url param was found.'}

In [85]:
import requests

url = "https://api.rpgsessions.com/import/emporium"
payload = { "data": "your data here" }
headers = {
    "Authorization": "Bearer your_token_here",
}

# Open the JSON file in binary mode
with open('creature.json', 'rb') as f:
    # Create a dictionary that will be sent as the files parameter
    files = {'file': ('creature.json', f)}

    # Make the request
    response = session.post(url, headers=headers, files=files)

print(response.json())

[{'name': 'Pie Goblin', 'id': '6490ab644fbaea0966bfb429'}]
